# Analyze sessions in batch from Phase 1 of AdaDrive (work in progress)

In [1]:
path_to_base_package = '../..'

import sys
from pathlib import Path

# setting path
sys.path.append(f"{path_to_base_package}")
import mne
import matplotlib
from mna.utils.rnapp_data_format import read_all_lslpresets, return_metadata_from_name, event_data_from_data
import pickle, os
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display
from scipy.io import savemat
import scipy
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from mna.utils.rnapp_data_format import read_all_lslpresets, return_metadata_from_name, event_data_from_data
import pickle
from statannotations.Annotator import Annotator
from collections import defaultdict
from scipy import stats
import mne
import glob 
import random
import re
import statsmodels.formula.api as smf
import statsmodels as sm
from scipy.stats import spearmanr
from mna.utils.data_access import *
from mna.utils.analysis import *
# matplotlib.use('Qt5Agg')
%matplotlib inline
plt.style.use('default')
from mna.sessions.eeg_session import process_session_eeg
from mna.utils.rnapp_data_format import read_all_files
# 1. Read a RN App, converted pkl file, and create the metadata and data structure

# Aux functions, read files

In [2]:
output_dir = f"{path_to_base_package}/output/batch_analysis_motor/"
output_dir_non_baseline_non_average = f"{path_to_base_package}/output/batch_analysis_motor/saved_files_non_baseline_non_average/" # saved files directory (trial or motor)
remove_sessions = [(15,1),(22,1)]
p_val_criteria = 0.05
preturn = 1000
rel_regions = {'premotor_regions': ['FC3', 'FC1', 'FCz', 'FC2', 'FC4'], 'dorsolateral_prefrontal': ['AF3', 'AFz', 'AF4'], 'intermediate_frontal': ['F3', 'F1', 'Fz', 'F2', 'F4']}
all_regions = sum(rel_regions.values(),[])

pupil_df = pd.read_csv(f"{path_to_base_package}/output/pupil_exposure/participant_level_exposure_fits.csv")
trial_dfs = pd.read_csv(f"{output_dir}all_results.csv")
motor_dfs = read_motor_csvs(output_dir)
motor_dfs['post_steer_event_raw'] = motor_dfs['post_steer_event_raw'].apply(str_list_to_list)
motor_epochs = get_motor_epochs(output_dir_non_baseline_non_average)

reading participant-level motor data
found cleaned epochs


# Clean up dfs

In [3]:
# seaborn
sns.set(font_scale=1.2)
sns.set_palette("tab10")
from mna.utils.batch_feature_extraction import clean_up_adadrive_trials

motor_outlier_cols = ['abs_sum_delta_steer_input']
cols_to_outlier_detect = ['bpm', 'sdnn', 'rmssd', 'pnn50']
experimental_cols = ['spoken_difficulty', 'trial_duration', 'density', 'trial_damage']
eye_cols = ['Left Pupil Diameter', "NSLR_count_Fixation", "NSLR_count_Saccade",
            'NSLR_mean_duration_Fixation', 'NSLR_mean_duration_Saccade',
            'NSLR_first_onset_Fixation', 'NSLR_first_onset_Saccade']
ecg_cols = ['bpm', 'sdnn', 'rmssd', 'pnn50']  # rmssd = parasympathetic
motor_cols = ['abs_sum_delta_steer_input', 'abs_sum_delta_brake_input', 'abs_sum_delta_throttle_input']

def remove_motor_overlaps(test_df):
    test_df = test_df.reset_index(drop=True)
    trial_list = list(zip(test_df.trial_start_time, test_df.trial_end_time, test_df.index))
    overlaps = []
    for i in range(1,len(trial_list)):
        base_data = trial_list[i]
        check_data = trial_list[i-1]
        if base_data[1] > check_data[0] and base_data[1] <= check_data[1]:
            overlaps.append((base_data, check_data, base_data[1]-check_data[0], base_data[2], check_data[2]))
            assert base_data[1]-check_data[0] != 1, 'Major issue, repeating trials found. Double check'
    drop_rows = []
    for overlap in overlaps:
        row_1 = test_df.iloc[overlap[-1]]
        row_2 = test_df.iloc[overlap[-2]]
        if np.argmax([row_1.Abs_Steer_Wheel_Degree, row_2.Abs_Steer_Wheel_Degree]) == 0: # if row 1 has larger steer motion, drop the other
            drop_rows.append(overlap[-2])
        else:
            drop_rows.append(overlap[-1])
    test_df = test_df.drop(drop_rows,axis=0)
    return test_df
    
def clean_up_trials(input_df):
    all_dfs_final = clean_up_adadrive_trials(input_df.copy())
    # damage change
    all_dfs_final = all_dfs_final.sort_values(by=['ppid', 'session', 'block', 'trial'])
    # nan, outliers
    #for col in motor_outlier_cols:
    #    all_dfs_final[col] = all_dfs_final[col].mask(all_dfs_final[col].sub(all_dfs_final[col].mean()).div(all_dfs_final[col].std()).abs().gt(2))
    #all_dfs_final['abs_sum_delta_brake_input'] = all_dfs_final['abs_sum_delta_brake_input'].mask(all_dfs_final['abs_sum_delta_brake_input']>.1)

    all_dfs_final['NSLR_first_onset_Fixation'] = all_dfs_final['NSLR_first_onset_Fixation'] - all_dfs_final[
        'trial_start_time']
    all_dfs_final['NSLR_first_onset_Saccade'] = all_dfs_final['NSLR_first_onset_Saccade'] - all_dfs_final[
        'trial_start_time']

    all_dfs_final[
        'throttle_over_brake'] = all_dfs_final.abs_sum_delta_throttle_input / all_dfs_final.abs_sum_delta_brake_input
    return all_dfs_final


trial_dfs = clean_up_trials(trial_dfs)
trial_dfs = trial_dfs.loc[~trial_dfs.ppid_session.isin([f"{es[0]}_{es[1]}" for es in remove_sessions])]
motor_dfs = clean_up_trials(motor_dfs)
print(f"removing ovlerlapping motor trials, starting epoch count {len(motor_dfs)}")
motor_dfs = remove_motor_overlaps(motor_dfs)
print(f"post removal epoch count {len(motor_dfs)}")
# luminance effect removal from pupil diameter
trial_dfs['Raw Left Pupil Diameter'] = trial_dfs['Left Pupil Diameter']
motor_dfs['Raw Left Pupil Diameter'] = motor_dfs['Left Pupil Diameter']
trial_dfs = trial_dfs.reset_index(drop=True)
adjustments=[]
for index, row in trial_dfs.iloc[1:].iterrows():
    last_ppid = trial_dfs.iloc[index - 1].ppid
    last_session = trial_dfs.iloc[index - 1].session
    last_trial = trial_dfs.iloc[index - 1].trial
    last_opacity = trial_dfs.iloc[index - 1].density
    if ((row.ppid == last_ppid) & (row.session == last_session) & (row.trial == last_trial + 1)):  # if continuous
        # if there is a significant effect of opacity on pupil
        if pupil_df.loc[pupil_df['sub'] == last_ppid, 'p_opacities'].values < p_val_criteria:
            this_opacity = row.density
            this_pupil_diameter = row['Left Pupil Diameter']
            weight = pupil_df.loc[pupil_df['sub'] == last_ppid, 'w_opacities']
            adjustment = ((this_opacity - last_opacity) * weight).values[0]
            trial_dfs.iloc[index, trial_dfs.columns.get_loc('Left Pupil Diameter')] += adjustment
            # this needs to be converted to array b/c of pandas issues
            old_pupil_value = np.array(motor_dfs.loc[(motor_dfs.ppid == last_ppid) & (motor_dfs.session == last_session) & (
                        motor_dfs.trial == last_trial + 1), 'Left Pupil Diameter']) 
            motor_dfs.loc[(motor_dfs.ppid == last_ppid) & (motor_dfs.session == last_session) & (
                        motor_dfs.trial == last_trial + 1), 'Left Pupil Diameter'] = (old_pupil_value-adjustment).T  # update motor df too
            # do also for motor_epochs
            old_pupil_value = motor_epochs.metadata.loc[(motor_epochs.metadata.ppid == last_ppid) &
                                      (motor_epochs.metadata.session == last_session) &
                                      (motor_epochs.metadata.trial == last_trial + 1), 'Left Pupil Diameter']
            motor_epochs.metadata.loc[(motor_epochs.metadata.ppid == last_ppid) &
                                      (motor_epochs.metadata.session == last_session) &
                                      (motor_epochs.metadata.trial == last_trial + 1), 'Left Pupil Diameter'] = (old_pupil_value-adjustment).T
# pupil bins
motor_dfs['pupil_bin'] = motor_dfs.groupby(['ppid'])['Left Pupil Diameter'].transform(
    lambda x: pd.qcut(x, 2, labels=['low', 'high']))
trial_dfs['pupil_bin'] = trial_dfs.groupby(['ppid'])['Left Pupil Diameter'].transform(
    lambda x: pd.qcut(x, 2, labels=['low', 'high']))
motor_epochs.metadata['pupil_bin'] = motor_epochs.metadata.groupby(['ppid'])['Left Pupil Diameter'].transform(
    lambda x: pd.qcut(x, 2, labels=['low', 'high']))
motor_dfs['pupil_bin_encoded'] = motor_dfs.groupby(['ppid'])['Left Pupil Diameter'].transform(
    lambda x: pd.qcut(x, 2, labels=[0, 1]))
trial_dfs['pupil_bin_encoded'] = trial_dfs.groupby(['ppid'])['Left Pupil Diameter'].transform(
    lambda x: pd.qcut(x, 2, labels=[0, 1]))
motor_epochs.metadata['pupil_bin_encoded'] = motor_epochs.metadata.groupby(['ppid'])['Left Pupil Diameter'].transform(
    lambda x: pd.qcut(x, 2, labels=[0, 1]))


# participant-level binning of motor data, replaces the session-level info already there
motor_dfs = get_motor_intensity_info(motor_dfs)
motor_epochs.metadata = get_motor_intensity_info(motor_epochs.metadata)


# ensure that epochs that removed from motor epochs are also removed from EEG analysis (we don't do the reverse since we have non-eeg, usable data)
df_all = motor_epochs.metadata[['ppid', 'session', 'block', 'trial']].merge(motor_dfs[['ppid', 'session', 'block', 'trial']].drop_duplicates(), on=['ppid', 'session', 'block', 'trial'], 
                   how='inner', indicator=True)
i1 = motor_epochs.metadata.set_index(['ppid', 'session', 'block', 'trial']).index
i2 = df_all.set_index(['ppid', 'session', 'block', 'trial']).index
motor_epochs = motor_epochs[i1.isin(i2)]
# motor_epochs.set_eeg_reference('average') # custom eeg reference is not allowed for MNE source modeling
motor_epochs.apply_baseline((-((preturn+250) / 1000), -((preturn) / 1000)))

removing ovlerlapping motor trials, starting epoch count 6291
post removal epoch count 6218
Replacing existing metadata with 75 columns
Applying baseline correction (mode: mean)


Number of events,5126
Events,easy: 1637hard: 1132unknown: 2357
Time range,-1.250 – 0.000 sec
Baseline,-1.250 – -1.000 sec


# Export to MATLAB

In [4]:
def get_files_for_pp(x):
    all_pps_files = [file_to_sess[v] for v in file_to_sess if file_to_sess[v][0] == x]
    all_pps_files.sort(key = lambda x: x[1])
    sorted_files = [next((filename for filename, pp_sess in file_to_sess.items() if pp_sess == apf), None) for apf in all_pps_files]
    return sorted_files

data_dir = f"{path_to_base_package}/data/"
lsl_dir = f"{path_to_base_package}/mna/LSLPresets/"
vid_dir = f"{path_to_base_package}/data/videos/"
output_path = f"{path_to_base_package}/output/matlab_exports/"
Path(output_path).mkdir(parents=True, exist_ok=True)

onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and '.pkl' in f]
file_to_sess = {f: (int(f.rsplit('Sbj_',1)[1].split('-')[0]),int(f.rsplit('Ssn_',1)[1].split('.')[0])) for f in onlyfiles}
all_pps = list(set([file_to_sess[v][0] for v in file_to_sess]))

In [5]:
def get_even_trials(input_df, interval_length = 5):
    input_df = input_df[['trial_start_time', 'trial_end_time', 'spoken_difficulty_encoded']]
    trial_starts = []
    spoken_diffs = []
    for idx, row in input_df.iterrows():
        range_trial_starts = (row.trial_start_time + np.arange(0,(row.trial_end_time - row.trial_start_time),interval_length))
        trial_starts.append(range_trial_starts[:-1])
        spoken_diffs.append([row.spoken_difficulty_encoded]*(len(range_trial_starts)-1))
    epoched_trial_starts = pd.DataFrame(np.concatenate(trial_starts),columns=['trial_start_time'])
    epoched_trial_starts['spoken_difficulty_encoded'] = np.concatenate(spoken_diffs)
    return epoched_trial_starts

In [6]:
interval_length = 5
# REFERENCE ICA
input_path = data_dir + '09_14_2022_13_17_39-Exp_adadrive-Sbj_20-Ssn_3.dats.pkl'
print(f"input_path {input_path}")
metadata_jsons = read_all_lslpresets(path_to_jsonfiles=lsl_dir)
with open(input_path, 'rb') as handle:
    rns_data = pickle.load(handle)

for key in rns_data.keys():
    rns_data[key].append(return_metadata_from_name(key, metadata_jsons))

eeg_channel_names = mne.channels.make_standard_montage('biosemi64').ch_names
eeg_df = pd.DataFrame(rns_data['BioSemi'][0], columns=rns_data['BioSemi'][1],
                    index=rns_data['BioSemi'][2]['ChannelNames']).T
eeg_df = eeg_df.iloc[:,1:65]
eeg_df.columns = eeg_channel_names

sub_trials = trial_dfs[(trial_dfs.ppid == 20) & (trial_dfs.session == 3)]
epoched_trial_starts = get_even_trials(sub_trials, interval_length = interval_length)
post_processed_event_df, epochs, events, info, reject_log, ref_ica, eog_idx = process_session_eeg(rns_data, epoched_trial_starts,
                        run_autoreject=True, run_ica=True, save_raw_eeg = False, sbj_session = 'sbj20ssn03', 
                        template_ica = None, analyze_pre_ica = False, filter_events=False,tmax = interval_length)

input_path ../../data/09_14_2022_13_17_39-Exp_adadrive-Sbj_20-Ssn_3.dats.pkl
Creating RawArray with float64 data, n_channels=89, n_times=3399947
    Range : 0 ... 3399946 =      0.000 ...  1660.130 secs
Ready.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 55.00 Hz
- Upper trans

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.8s finished


Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 64 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 32.5s.
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (64 components)
    Zeroing out 2 ICA components
    Projecting back using 64 PCA components
Adding metadata with 2 columns
198 matching events found
Setting baseline interval to [-0.203125, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 198 events and 667 original time points ...
0 bad epochs dropped
Dropped 46 epochs: 18, 20, 28, 34, 37, 41, 42, 50, 51, 56, 63, 71, 72, 75, 76, 78, 89,

In [58]:
# NOTE: this finds the minimum trial per session to
for pp in all_pps[3:]:
    print(f"participant {pp}")
    all_eeg_trials = []
    all_motor_trials = []
    labels = []
    for selected_file in get_files_for_pp(pp):
        selected_pp_sess = file_to_sess[selected_file]
        input_path = data_dir + selected_file # pick a random file, idx 26 and

        print(f"input_path {input_path}")
        metadata_jsons = read_all_lslpresets(path_to_jsonfiles=lsl_dir)
        with open(input_path, 'rb') as handle:
            rns_data = pickle.load(handle)

        for key in rns_data.keys():
            rns_data[key].append(return_metadata_from_name(key, metadata_jsons))

        eeg_channel_names = mne.channels.make_standard_montage('biosemi64').ch_names
        eeg_df = pd.DataFrame(rns_data['BioSemi'][0], columns=rns_data['BioSemi'][1],
                          index=rns_data['BioSemi'][2]['ChannelNames']).T
        eeg_df = eeg_df.iloc[:,1:65]
        eeg_df.columns = eeg_channel_names

        motor_df = pd.DataFrame(rns_data['Unity_MotorInput'][0], columns=rns_data['Unity_MotorInput'][1],
                                  index=rns_data['Unity_MotorInput'][2]['ChannelNames']).T

        sub_trials = trial_dfs[(trial_dfs.ppid == selected_pp_sess[0]) & (trial_dfs.session == selected_pp_sess[1])]
        if len(sub_trials)>0:
            epoched_trial_starts = get_even_trials(sub_trials, interval_length = interval_length)
            post_processed_event_df, epochs, events, info, reject_log, ref_ica, eog_idx  = process_session_eeg(rns_data, epoched_trial_starts,
                            run_autoreject=True, run_ica=True, save_raw_eeg = False, sbj_session = str(selected_pp_sess[0])+'_'+str(selected_pp_sess[1]), 
                            template_ica = ref_ica, analyze_pre_ica = False, filter_events=False,tmax = interval_length)

            if len(post_processed_event_df)==0:
                print('This session has no usable trials.')
                continue
            post_processed_event_df['trial_end_time'] = post_processed_event_df.trial_start_time+interval_length
            post_processed_event_df['trial_duration'] = interval_length
            max_eeg_trial = int(post_processed_event_df.trial_duration.min()*2048) # ensure same size
            max_motor_trial = int(post_processed_event_df.trial_duration.min()*40) # ensure same size
            new_sample_rate = 128
            epochs_metadata_df = epochs.metadata.reset_index(drop=True)
            for index,trial in epochs_metadata_df.iterrows():
                # if we want to use start of each trial
                #sub_eeg_df = eeg_df[(eeg_df.index >= trial.trial_start_time) & (eeg_df.index <= trial.trial_end_time)].iloc[:max_eeg_trial]
                #sub_motor_df = motor_df[(motor_df.index >= trial.trial_start_time) & (motor_df.index <= trial.trial_end_time)].iloc[:max_motor_trial]

                # maximize the data through splitting trial data
                sub_trial_motor = motor_df[(motor_df.index >= trial.trial_start_time) & (motor_df.index <= trial.trial_start_time+interval_length)]

                secs = sub_trial_motor.shape[0]/40 # Number of seconds in signal X
                samps = int(secs*new_sample_rate)     # Number of samples to downsample
                sub_trial_motor = scipy.signal.resample(sub_trial_motor, samps).T
                all_eeg_trials.append(epochs[index].get_data().squeeze())
                all_motor_trials.append(sub_trial_motor)
                labels.append(f"{selected_pp_sess[0]}_{selected_pp_sess[1]}")
    if len(all_eeg_trials)>0:
        shortest_session_trial_length = min([a.shape[1] for a in all_eeg_trials])
        trimmed_trials = [a[:,:shortest_session_trial_length] for a in all_eeg_trials]
        all_eeg_trials = np.array(trimmed_trials)

        shortest_session_trial_length = min([a.shape[1] for a in all_motor_trials])
        trimmed_trials = [a[:,:shortest_session_trial_length] for a in all_motor_trials]
        all_motor_trials = np.array(trimmed_trials)
        print(f"Saved a total of {all_eeg_trials.shape[0]} trials")
        mdic = {"eeg": all_eeg_trials, "motor": all_motor_trials, "labels": labels, "ppid": pp,"sample_rate": new_sample_rate}
        savemat(f"{output_path}{selected_pp_sess[0]}.mat", mdic)
    else:
        print('No trials found for this participant.')

participant 15
input_path ../../data/09_07_2022_13_57_15-Exp_adadrive-Sbj_15-Ssn_1.dats.pkl
No trials found for this participant.
participant 16
input_path ../../data/09_12_2022_14_43_23-Exp_adadrive-Sbj_16-Ssn_1.dats.pkl
Creating RawArray with float64 data, n_channels=89, n_times=3174654
    Range : 0 ... 3174653 =      0.000 ...  1550.124 secs
Ready.
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.0s finished


Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 64 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 64.1s.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (64 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components
Adding metadata with 2 columns
151 matching events found
Setting baseline interval to [-0.203125, 0.0] sec

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    2.8s finished


Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 64 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 63.5s.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (64 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components
Adding metadata with 2 columns
136 matching events found
Setting baseline interval to [-0.203125, 0.0] sec

...

Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 64 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 40.5s.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (64 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components
Adding metadata with 2 columns
119 matching events found
Setting baseline interval to [-0.203125, 0.0] sec

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    2.8s finished


Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 64 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 47.8s.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Median correlation with constructed map: 0.964
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (64 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components
Adding metadata with 2 columns
147 matching events found
Setting baseline interval to [-0.203125, 0.0] sec

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    3.0s finished


Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 64 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 59.4s.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Median correlation with constructed map: 1.000
Displaying selected ICs per subject.
No maps selected for subject [1], consider a more liberal threshold.
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (64 components)
    Zeroing out 0 ICA components
    Projecting back using 64 PCA components
Adding metadata with 2 columns
134 matching events found
Setting baseline interval to [-0.203125, 0.0] sec